In [1]:
import pandapower as pp

In [2]:
import pyomo.environ as pe

In [3]:
import math

In [4]:
import random

In [5]:
net = pp.create_empty_network() 
b1 = pp.create_bus(net, vn_kv=13.2)
b2 = pp.create_bus(net, vn_kv=13.2)
b3 = pp.create_bus(net, vn_kv=13.2)
b4 = pp.create_bus(net, vn_kv=13.2)

pp.create_line(net, from_bus=b1, to_bus=b2, length_km=0.8, std_type="NAYY 4x50 SE")
pp.create_line(net, from_bus=b2, to_bus=b3, length_km=1.2, std_type="NAYY 4x50 SE")
pp.create_line(net, from_bus=b3, to_bus=b4, length_km=1.0, std_type="NAYY 4x50 SE")

pp.create_ext_grid(net, bus=b1)

pp.create_load(net, bus=b3, p_mw=0.350)

0

In [6]:
 pp.runpp(net)

In [7]:
print(net.res_bus.vm_pu)
print(net.res_line)

0    1.000000
1    0.998977
2    0.997436
3    0.997439
Name: vm_pu, dtype: float64
      p_from_mw  q_from_mvar       p_to_mw     q_to_mvar         pl_mw  \
0  3.509116e-01    -0.034250 -3.505460e-01  2.511060e-02  3.655632e-04   
1  3.505460e-01    -0.025111 -3.500001e-01  1.143635e-02  5.459089e-04   
2  1.210575e-07    -0.011436  3.942859e-11 -3.873803e-10  1.210970e-07   

    ql_mvar  i_from_ka       i_to_ka      i_ka  vm_from_pu  va_from_degree  \
0 -0.009140   0.015421  1.538744e-02  0.015421    1.000000        0.000000   
1 -0.013674   0.015387  1.535608e-02  0.015387    0.998977       -0.012683   
2 -0.011436   0.000501  1.707477e-11  0.000501    0.997436       -0.028839   

   vm_to_pu  va_to_degree  loading_percent  
0  0.998977     -0.012683        10.860096  
1  0.997436     -0.028839        10.836226  
2  0.997439     -0.030053         0.353167  


In [8]:
pp.create_sgen(net, b2, p_mw=0.05, q_mvar=0.025, max_p_mw=0.2, max_q_mvar=0.2)
pp.create_sgen(net, b4, p_mw=0.07, q_mvar=0.025, max_p_mw=0.3, max_q_mvar=0.2)


1

In [9]:
pp.runpp(net)

In [10]:
print(net.res_bus.vm_pu)
print(net.res_line)

0    1.000000
1    0.999351
2    0.998135
3    0.998408
Name: vm_pu, dtype: float64
   p_from_mw  q_from_mvar   p_to_mw  q_to_mvar     pl_mw   ql_mvar  i_from_ka  \
0   0.230553    -0.084334 -0.230378   0.075166  0.000175 -0.009168   0.010738   
1   0.280378    -0.050166 -0.280022   0.036453  0.000356 -0.013714   0.012466   
2  -0.069978    -0.036453  0.070000   0.025000  0.000022 -0.011453   0.003458   

    i_to_ka      i_ka  vm_from_pu  va_from_degree  vm_to_pu  va_to_degree  \
0  0.010606  0.010738    1.000000        0.000000  0.999351     -0.018512   
1  0.012374  0.012466    0.999351       -0.018512  0.998135     -0.038710   
2  0.003256  0.003458    0.998135       -0.038710  0.998408     -0.043302   

   loading_percent  
0         7.561657  
1         8.779012  
2         2.434919  


In [11]:
print(net)

This pandapower network includes the following parameter tables:
   - bus (4 elements)
   - load (1 element)
   - sgen (2 elements)
   - ext_grid (1 element)
   - line (3 elements)
 and the following results tables:
   - res_bus (4 elements)
   - res_line (3 elements)
   - res_ext_grid (1 element)
   - res_load (1 element)
   - res_sgen (2 elements)


In [12]:
net.sgen

,name,bus,p_mw,q_mvar,sn_mva,scaling,in_service,type,current_source,max_p_mw,max_q_mvar
0,None,1,0.05,0.025,NaN,1.0,True,wye,True,0.2,0.2
1,None,3,0.07,0.025,NaN,1.0,True,wye,True,0.3,0.2


Agregamos los parámetros del modelo a los elementos de la red.

In [13]:
def net_add_optfw(net):
    #agregamos parámetros para el costo inicial en forma de un valor constante ic_0 y un valor función lineal de la potencia ic_1
    net.ext_grid['ic_0_mu'] = 0.0
    net.ext_grid['ic_1_mu'] = 0.0

    #idem para costo operativo
    net.ext_grid['oc_0_mu'] = 0.0
    net.ext_grid['oc_1_mu'] = 0.0

    #potencia disponible
    net.ext_grid['ap_mw'] = 0.5
    
    #potencia entregada
    net.ext_grid['op_mw'] = 0.0
    
    #las restricciones pueden ser ninguna, o una lista de restricciones tipo pyomo
    net.ext_grid['constraints'] = None

    #se repite para la carga y los generadores estáticos
    net.load['ic_0_mu'] = 0.0
    net.load['ic_1_mu'] = 0.0
    net.load['oc_0_mu'] = 0.0
    net.load['oc_1_mu'] = 0.0
    net.load['ap_mw'] = 0.5
    net.load['op_mw'] = 0.0
    net.load['constraints'] = None

    net.sgen['ic_0_mu'] = 0.0
    net.sgen['ic_1_mu'] = 0.0
    net.sgen['oc_0_mu'] = 0.0
    net.sgen['oc_1_mu'] = 0.0
    net.sgen['ap_mw'] = 0.5
    net.sgen['op_mw'] = 0.0
    net.sgen['constraints'] = None

In [14]:
net_add_optfw(net)

Vemos como queda por ejemplo el elemento de red externa:

In [15]:
net.ext_grid

,name,bus,vm_pu,va_degree,in_service,ic_0_mu,ic_1_mu,oc_0_mu,oc_1_mu,ap_mw,op_mw,constraints
0,None,0,1.0,0.0,True,0.0,0.0,0.0,0.0,0.5,0.0,None


Defino una función normal de Python para que nos va a dar la componente lineal del costo de la energía

Variables que define el modelo:

y para el año

d para el día del año

h para la hora del día

dt la granularidad del modelo, en horas


temp la temperatura

wv la velocidad del viento

I la irradiación solar


eg el crecimiento económico en pu respecto al año 0


Estas variables llegan como parámetros en forma de diccionario en el argumento 'model_status' (ver este nombre)

Todas las funciones reciben este argumento, la lógica de la función indica que valor retorna. Pr ejemplo, si la granularidad es 24 h, debe retornar el valor medio del parámetro producido.

In [16]:
def oc_1_ext_grid(model_status={}):
    #modelo sencillo con dos precios, uno entre 0 a 18 y otro de 18 a 24
    res = 0.0
    sx = 1e-6
    if 'h' in model_status:
        h = model_status['h']
        if 0.0 <= h and h < 18.0:
            res = 3600.0*sx
        elif 18 <= h and h < 24.0:
            res = 5400.0*sx
        else:
            raise ValueError("Hour outside model range")
    else:
        raise ValueError("Hour not defined")
    
    return res

In [17]:
#test:
#m_s = {'y': 0, 'd': 180, 'h': 14.0, 'dt': 1.0, 'temp': 12.0, 'wv': 10.0, 'eg': 1.0}

#oc_1_ext_grid(m_s)

In [18]:
#20 hs: 5400
#m_s['h'] = 20.5
#oc_1_ext_grid(m_s)

In [19]:
#25 hs: error
#m_s['h'] = 25
#oc_1_ext_grid(m_s)

In [20]:
net.ext_grid['oc_1_mu'][0] = oc_1_ext_grid

In [21]:
net.ext_grid

,name,bus,vm_pu,va_degree,in_service,ic_0_mu,ic_1_mu,oc_0_mu,oc_1_mu,ap_mw,op_mw,constraints
0,None,0,1.0,0.0,True,0.0,0.0,0.0,<function oc_1_ext_grid at 0x0000015EAFA56AF0>,0.5,0.0,None


In [22]:
def demand(model_status={}):
    #modelo sencillo en forma de escalones
    #devuelvr la fracción de la carga empleada
    res = 0.0
    if 'h' in model_status:
        h = model_status['h']
        if 0.0 <= h and h < 6.0:
            res = 0.2
        elif 6.0 <= h and h < 8.0:
            res = 0.4
        elif 8.0 <= h and h < 18.0:
            res = 0.5
        elif 18.0 <= h and h < 22.0:
            res = 1.0
        elif 22.0 <= h and h < 24.0:
            res = 0.3            
        else:
            raise ValueError("Hour outside model range")
    else:
        raise ValueError("Hour not defined")
    
    return res

In [23]:
net.load['op_mw'][0] = demand

In [24]:
net.load

,name,bus,p_mw,q_mvar,const_z_percent,const_i_percent,sn_mva,scaling,in_service,type,ic_0_mu,ic_1_mu,oc_0_mu,oc_1_mu,ap_mw,op_mw,constraints
0,None,2,0.35,0.0,0.0,0.0,NaN,1.0,True,wye,0.0,0.0,0.0,0.0,0.5,<function demand at 0x0000015EC17FA940>,None


In [25]:
def solar_output(model_status={}):
    #modelo sencillo
    #devuelvr la fracción de la potencia entregada en función del tiempo
    #no toma en cuenta la radicación solar ni la temperatura
    res = 0.0
    if 'h' in model_status:
        h = model_status['h']
        if 0.0 <= h and h < 8.0:
            res = 0.0
        elif 8.0 <= h and h < 18.0:
            res = math.exp(-(h-13.0)**2/8)
        elif 18.0 <= h and h < 24.0:
            res = 0.0
        else:
            raise ValueError("Hour outside model range")
    else:
        raise ValueError("Hour not defined")
    
    return res

In [26]:
#test:
#m_s = {'y': 0, 'd': 180, 'h': 43.0, 'dt': 1.0, 'temp': 12.0, 'wv': 10.0, 'eg': 1.0}
#solar_output(m_s)

In [27]:
net.sgen['name'][0] = 'PV1'
net.sgen['ap_mw'][0] = solar_output
net.sgen['ic_0_mu'][0] = 2
net.sgen['ic_1_mu'][0] = 3*0.75
net.sgen['oc_0_mu'][0] = 0.0
net.sgen['oc_1_mu'][0] = 500e-6


In [79]:
def wind_output(model_status={}):
    #modelo sencillo
    #devuelvr la fracción de la potencia entregada en función del tiempo
    #no toma en cuenta la radicación solar ni la temperatura
    res = 0.0
    if 'wv' in model_status:
        wv = model_status['wv']
        if 0.0 <= wv and wv < 3.0:
            res = 0.0
        elif 3.0 <= wv and wv < 15.0:
            res = (wv-3.0)/(15.0-3.0)
        elif 15.0 <= wv and wv < 25.0:
            res = 1.0
        else:
            res = 0.0
    else:
        raise ValueError("Wind Velocity not defined")
    
    return res

In [109]:
net.sgen['name'][1] = 'WT1'
net.sgen['ap_mw'][1] = wind_output
net.sgen['ic_0_mu'][1] = 100.0
net.sgen['ic_1_mu'][1] = 3.0
net.sgen['oc_0_mu'][1] = 0.0
net.sgen['oc_1_mu'][1] = 700e-6

In [110]:
net.sgen

,name,bus,p_mw,q_mvar,sn_mva,scaling,in_service,type,current_source,max_p_mw,max_q_mvar,ic_0_mu,ic_1_mu,oc_0_mu,oc_1_mu,ap_mw,op_mw,constraints
0,PV1,1,0.05,0.025,NaN,1.0,True,wye,True,0.2,0.2,2.0,2.25,0.0,0.0005,<function solar_output at 0x0000015EC1979040>,0.0,None
1,WT1,3,0.07,0.025,NaN,1.0,True,wye,True,0.3,0.2,100.0,3.00,0.0,0.0007,<function wind_output at 0x0000015EC19ED040>,0.0,None


Primero construyo un modelo manualmente:

Es un modelo de dimensionamiento de 2 DERs (WT+PV)

In [111]:
m = pe.ConcreteModel()

In [112]:
dias = 365
años = 5
M = 1e3

In [113]:
#el rango de horas:
T_i = range(24)

In [114]:
#las variables que definien si construir y que potencia
m.ap_mw_PV = pe.Var(within = pe.NonNegativeReals)  #Tamano PV
m.ap_mw_WT = pe.Var(within = pe.NonNegativeReals)  #Tamano WT

m.sgen_PV = pe.Var(within = pe.Binary)
m.sgen_WT = pe.Var(within = pe.Binary) 

In [115]:
#despacho de solar y WT disponible
m.p_mw_PV = pe.Var(T_i, within = pe.NonNegativeReals)
m.p_mw_WT = pe.Var(T_i, within = pe.NonNegativeReals)

In [116]:
#energia comprada a la red
m.p_mw_Ext = pe.Var(T_i, within = pe.NonNegativeReals, bounds = (0, net.ext_grid['ap_mw'][0]))
#si puedo exportar:
##m.p_mw_Ext = pe.Var(T_i, bounds = (-net.ext_grid['ap_mw'][0], net.ext_grid['ap_mw'][0]))

In [117]:
#el rango de condiciones:
m_s = {'y': 0, 'd': 180, 'h': 43.0, 'dt': 1.0, 'temp': 12.0, 'wv': 10.0, 'eg': 1.0}

Escenarios_i = []
for t in T_i:
    Escenarios_i.append({'y': 0, 'd': 180, 'h': 1.0*t, 'dt': 1.0, 'temp': 12.0, 'wv': random.uniform(1.0, 20.0), 'eg': 1.0})


In [118]:
#Escenarios_i

In [119]:
#los costos iniciales:
costos_iniciales = net.ext_grid['ic_0_mu'][0] + net.load['ic_0_mu'][0] + m.sgen_PV*net.sgen['ic_0_mu'][0] + m.sgen_WT*net.sgen['ic_0_mu'][1] +  \
                   net.ext_grid['ic_1_mu'][0]*0 + net.load['ic_1_mu'][0]*0 + m.ap_mw_PV * net.sgen['ic_1_mu'][0] + m.ap_mw_WT * net.sgen['ic_1_mu'][1]

In [120]:
print(costos_iniciales)

2.0*sgen_PV + 100.0*sgen_WT + 2.25*ap_mw_PV + 3.0*ap_mw_WT


In [121]:
costos_variables = sum(m.p_mw_PV[t]*net.sgen['oc_1_mu'][0]*dias*años+ m.p_mw_WT[t]*net.sgen['oc_1_mu'][1]*dias*años + m.p_mw_Ext[t]*net.ext_grid['oc_1_mu'][0](Escenarios_i[t])*dias*años for t in T_i)

In [122]:
print(costos_variables)

0.9125*p_mw_PV[0] + 1.2775*p_mw_WT[0] + 6.57*p_mw_Ext[0] + 0.9125*p_mw_PV[1] + 1.2775*p_mw_WT[1] + 6.57*p_mw_Ext[1] + 0.9125*p_mw_PV[2] + 1.2775*p_mw_WT[2] + 6.57*p_mw_Ext[2] + 0.9125*p_mw_PV[3] + 1.2775*p_mw_WT[3] + 6.57*p_mw_Ext[3] + 0.9125*p_mw_PV[4] + 1.2775*p_mw_WT[4] + 6.57*p_mw_Ext[4] + 0.9125*p_mw_PV[5] + 1.2775*p_mw_WT[5] + 6.57*p_mw_Ext[5] + 0.9125*p_mw_PV[6] + 1.2775*p_mw_WT[6] + 6.57*p_mw_Ext[6] + 0.9125*p_mw_PV[7] + 1.2775*p_mw_WT[7] + 6.57*p_mw_Ext[7] + 0.9125*p_mw_PV[8] + 1.2775*p_mw_WT[8] + 6.57*p_mw_Ext[8] + 0.9125*p_mw_PV[9] + 1.2775*p_mw_WT[9] + 6.57*p_mw_Ext[9] + 0.9125*p_mw_PV[10] + 1.2775*p_mw_WT[10] + 6.57*p_mw_Ext[10] + 0.9125*p_mw_PV[11] + 1.2775*p_mw_WT[11] + 6.57*p_mw_Ext[11] + 0.9125*p_mw_PV[12] + 1.2775*p_mw_WT[12] + 6.57*p_mw_Ext[12] + 0.9125*p_mw_PV[13] + 1.2775*p_mw_WT[13] + 6.57*p_mw_Ext[13] + 0.9125*p_mw_PV[14] + 1.2775*p_mw_WT[14] + 6.57*p_mw_Ext[14] + 0.9125*p_mw_PV[15] + 1.2775*p_mw_WT[15] + 6.57*p_mw_Ext[15] + 0.9125*p_mw_PV[16] + 1.2775*p_mw_WT[16

In [123]:
costos = costos_iniciales + costos_variables

In [124]:
print(costos)

2.0*sgen_PV + 100.0*sgen_WT + 2.25*ap_mw_PV + 3.0*ap_mw_WT + 0.9125*p_mw_PV[0] + 1.2775*p_mw_WT[0] + 6.57*p_mw_Ext[0] + 0.9125*p_mw_PV[1] + 1.2775*p_mw_WT[1] + 6.57*p_mw_Ext[1] + 0.9125*p_mw_PV[2] + 1.2775*p_mw_WT[2] + 6.57*p_mw_Ext[2] + 0.9125*p_mw_PV[3] + 1.2775*p_mw_WT[3] + 6.57*p_mw_Ext[3] + 0.9125*p_mw_PV[4] + 1.2775*p_mw_WT[4] + 6.57*p_mw_Ext[4] + 0.9125*p_mw_PV[5] + 1.2775*p_mw_WT[5] + 6.57*p_mw_Ext[5] + 0.9125*p_mw_PV[6] + 1.2775*p_mw_WT[6] + 6.57*p_mw_Ext[6] + 0.9125*p_mw_PV[7] + 1.2775*p_mw_WT[7] + 6.57*p_mw_Ext[7] + 0.9125*p_mw_PV[8] + 1.2775*p_mw_WT[8] + 6.57*p_mw_Ext[8] + 0.9125*p_mw_PV[9] + 1.2775*p_mw_WT[9] + 6.57*p_mw_Ext[9] + 0.9125*p_mw_PV[10] + 1.2775*p_mw_WT[10] + 6.57*p_mw_Ext[10] + 0.9125*p_mw_PV[11] + 1.2775*p_mw_WT[11] + 6.57*p_mw_Ext[11] + 0.9125*p_mw_PV[12] + 1.2775*p_mw_WT[12] + 6.57*p_mw_Ext[12] + 0.9125*p_mw_PV[13] + 1.2775*p_mw_WT[13] + 6.57*p_mw_Ext[13] + 0.9125*p_mw_PV[14] + 1.2775*p_mw_WT[14] + 6.57*p_mw_Ext[14] + 0.9125*p_mw_PV[15] + 1.2775*p_mw_WT[15]

In [125]:
m.value = pe.Objective(
expr = costos_iniciales + costos_variables,
sense = pe.minimize )

In [126]:
def hourly_power_balance(m, t):
    return m.p_mw_Ext[t] + m.p_mw_PV[t] + m.p_mw_WT[t] - net.load['op_mw'][0](Escenarios_i[t])*net.load['p_mw'][0] == 0

In [127]:
m.hourly_power_balance = pe.Constraint(T_i, rule = hourly_power_balance)

In [128]:
#dimensionamiento solar:
m.PV_dim = pe.Constraint(T_i, rule = (lambda m, t : m.p_mw_PV[t] <= m.ap_mw_PV*net.sgen['ap_mw'][0](Escenarios_i[t])))

In [129]:
m.PV_build = pe.Constraint( expr = m.ap_mw_PV <= m.sgen_PV*M)

In [130]:
#dimensionamiento WT:
m.WT_dim = pe.Constraint(T_i, rule = (lambda m, t : m.p_mw_WT[t] <= m.ap_mw_WT*net.sgen['ap_mw'][1](Escenarios_i[t])))

In [131]:
m.WT_build = pe.Constraint( expr = m.ap_mw_WT <= m.sgen_WT*M)

In [132]:
opt = pe.SolverFactory('glpk')

result_obj= opt.solve(m, tee=True)

GLPSOL: GLPK LP/MIP Solver, v4.65
Parameter(s) specified in the command line:
 --write C:\Users\jmsar\AppData\Local\Temp\tmpcz4chz31.glpk.raw --wglp C:\Users\jmsar\AppData\Local\Temp\tmpe7mjyxru.glpk.glp
 --cpxlp C:\Users\jmsar\AppData\Local\Temp\tmplinz65or.pyomo.lp
Reading problem data from 'C:\Users\jmsar\AppData\Local\Temp\tmplinz65or.pyomo.lp'...
C:\Users\jmsar\AppData\Local\Temp\tmplinz65or.pyomo.lp:542: warning: lower bound of variable 'x3' redefined
C:\Users\jmsar\AppData\Local\Temp\tmplinz65or.pyomo.lp:542: warning: upper bound of variable 'x3' redefined
75 rows, 77 columns, 156 non-zeros
2 integer variables, all of which are binary
544 lines were read
Writing problem data to 'C:\Users\jmsar\AppData\Local\Temp\tmpe7mjyxru.glpk.glp'...
537 lines were written
GLPK Integer Optimizer, v4.65
75 rows, 77 columns, 156 non-zeros
2 integer variables, all of which are binary
Preprocessing...
43 rows, 35 columns, 86 non-zeros
2 integer variables, all of which are binary
Scaling...
 A: mi

In [133]:
m.pprint()

6 Set Declarations
    PV_dim_index : Size=1, Index=None, Ordered=False
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :   24 : {0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23}
    WT_dim_index : Size=1, Index=None, Ordered=False
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :   24 : {0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23}
    hourly_power_balance_index : Size=1, Index=None, Ordered=False
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :   24 : {0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23}
    p_mw_Ext_index : Size=1, Index=None, Ordered=False
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :   24 : {0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23}
    p_mw_PV_index : Size=1, Index=None, Ordered=False
       